In [12]:
file = open('demofile.dots', 'r', encoding='UTF-8')

In [13]:
lines = file.readlines()

In [10]:
import re
declarator = r":[_A-Za-z][A-Za-z0-9]*:"
literals = r"\"[A-Za-z0-9\;\,\.\-\:\^\*\ ]*\""
functional_symbols = [
    {
        "name":"open_parentheses",
        "matcher":r"\("
    },
    {
        "name":"close_parentheses",
        "matcher":r"\)"
    },
    {
        "name":"open_curly",
        "matcher":r"\{"
    },
    {
        "name":"close_curly",
        "matcher":r"\}"
    }
]
variable_types = [
    {
        "name": "string",
        "converts_to":str,
        "matcher":r"string "
    },
    {
        "name": "int",
        "converts_to": int,
        "matcher": r"int "
    },
    {
        "name": "float",
        "converts_to": float,
        "mathcer": r"float "
    }
]



In [18]:
def filler_inLine(found_group, line):
    filler = ""
    for i in range(0,len(found_group)):
        filler+=" "
    line = line.replace(found_group,filler)
    return line

In [21]:
from operator import indexOf
import re
stack = []
for line in lines:
    found_literal = re.search(literals,line)
    if found_literal:
        stack.append(
            {
                'line': indexOf(lines,line),
                'position': found_literal.span(),
                'match': found_literal.group()
            }
        )
        line = filler_inLine(found_literal.group(), line)
    found_declarator = re.search(declarator,line)
    if found_declarator:
        stack.append(
            {
                'line': indexOf(lines,line),
                'position': found_declarator.span(),
                'match': found_declarator.group()
            }
        )
    for functional_symbol in functional_symbols:
        found_functional_symbol = re.search(functional_symbol["matcher"], line)
        if found_functional_symbol:
            stack.append(            {
                'line': indexOf(lines,line),
                'position': found_functional_symbol.span(),
                'match': found_functional_symbol.group()
            })
            
print(stack)

[{'line': 0, 'position': (0, 3), 'match': ':m:'}, {'line': 0, 'position': (3, 4), 'match': '('}, {'line': 0, 'position': (4, 5), 'match': ')'}, {'line': 0, 'position': (5, 6), 'match': '{'}, {'line': 1, 'position': (21, 38), 'match': '"Hello :m: World"'}, {'line': 2, 'position': (4, 7), 'match': ':p:'}, {'line': 2, 'position': (7, 8), 'match': '('}, {'line': 2, 'position': (13, 14), 'match': ')'}, {'line': 7, 'position': (0, 1), 'match': '}'}]


In [22]:
for item in stack:
    print(str(lines[item['line']])[item['position'][0]:item['position'][1]])

:m:
(
)
{
"Hello :m: World"
:p:
(
)
}
